In [1]:
import numpy as np
import datetime, time
from multiprocess import Process, Queue, cpu_count
import multiprocess as multip
import os, re, glob
import _pickle as cPickle
from PyQt5 import QtCore

import LaueTools.dict_LaueTools as dictLT
import LaueTools.IOLaueTools as IOLT
import LaueTools.generaltools as GT
import LaueTools.CrystalParameters as CP
import LaueTools.lauecore as LT
import LaueTools.LaueGeometry as Lgeo
import LaueTools.readmccd as RMCCD
import LaueTools.FitOrient as FitO
import LaueTools.findorient as FindO
from fast_histogram import histogram1d
import networkx as nx
import h5py
import time
import ipywidgets as widgets

tensorflow_keras = True
try:
    from keras.models import model_from_json
except:
    tensorflow_keras = False

from utils_lauenn import Symmetry, Lattice, SGLattice, chunker_list,\
                            rot_mat_to_euler, global_plots, save_sst

cpu_count_user = 8#cpu_count()
ncpu = cpu_count_user
total_progress = 0
#timermp1212 = QtCore.QTimer()
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
max_progress = 1

print("Number of CPUs available : ", ncpu)

-- OK! You are using python 3
Missing library libtiff, Please install: pylibtiff if you need open some tiff images
-- warning: module Image or PIL is not installed, but only used for templateimagematching
Number of CPUs available :  8


In [3]:
# =============================================================================
# USER INPUT
# =============================================================================
use_om_user = False
nb_spots_consider = 100
residues_threshold=0.5
nb_spots_global_threshold=8
option_global = "v2"
CST_ENERGYKEV = 12.398

material_ = "Al2TiO5"
material1_ = material_
symmetry_name = "orthorhombic"
symmetry1_name = symmetry_name
SG = 63

ubmat = 2
use_previous_UBmatrix_name = False
strain_calculation = True

model_direc = r"C:\Users\purushot\Desktop\pattern_matching\Al2TiO5_jan2022"

detectorfile = r"C:\Users\purushot\Desktop\Al2TiO5_laue\old_results\data\VF_P1\Ge_VF_P1\Ge_VF_P1.det"
emin = 5
emax = 21

filenameDirec = r"C:\Users\purushot\Desktop\Al2TiO5_laue\old_results\data\VF_P1\VF_P1_R1"
experimental_prefix = "VF_P1_R1_"
lim_x, lim_y = 21, 21

tolerance = 0.6
tolerance1 = tolerance

tolerance_strain = [0.6,0.55,0.5,0.45,0.4,0.35,0.3,0.25,0.2,0.15]
tolerance_strain1 = tolerance_strain

intensity_threshold = 50 #75 800
boxsize = 10
fit_peaks_gaussian = 1
FitPixelDev = 18
NumberMaxofFits = 2000 ### Max peaks per LP
bkg_treatment = "A-B"

softmax_threshold_global = 0.80 # softmax_threshold
mr_threshold_global = 0.90 # match rate threshold
cap_matchrate = 0.01 * 100 ## any UB matrix providing MR less than this will be ignored
coeff = 0.10 ## should be same as cap_matchrate or no?
coeff_overlap = 0.10 ##10% spots overlap to avoid bad orientation detection
material0_limit = 100000
material1_limit = 100000
material_phase_always_present = "none"
# =============================================================================
# END OF USER INPUT
# =============================================================================

In [4]:
max_progress = lim_x * lim_y

#timermp1212.setInterval(500) ## check every second (update the list of files in folder)
#timermp1212.timeout.connect(update_data_mp)
#timermp1212.start()

if symmetry_name =="cubic":
    material0_lauegroup = "11"
elif symmetry_name =="monoclinic":
    material0_lauegroup = "2"
elif symmetry_name == "hexagonal":
    material0_lauegroup = "9"
elif symmetry_name == "orthorhombic":
    material0_lauegroup = "3"
elif symmetry_name == "tetragonal":
    material0_lauegroup = "5"
elif symmetry_name == "trigonal":
    material0_lauegroup = "7"
elif symmetry_name == "triclinic":
    material0_lauegroup = "1"

material1_lauegroup = material0_lauegroup

a, b, c, alpha, beta, gamma = dictLT.dict_Materials[material_][1]

if symmetry_name =="cubic":
    crystal = SGLattice(int(SG), a)
    symmetry = Symmetry.cubic
    lattice_material = Lattice.cubic(a)
elif symmetry_name =="monoclinic":
    crystal = SGLattice(int(SG),a, b, c, beta)
    symmetry = Symmetry.monoclinic
    lattice_material = Lattice.monoclinic(a, b, c, beta)
elif symmetry_name == "hexagonal":
    crystal = SGLattice(int(SG),a, c)
    symmetry = Symmetry.hexagonal
    lattice_material = Lattice.hexagonal(a, c)
elif symmetry_name == "orthorhombic":
    crystal = SGLattice(int(SG),a, b, c)
    symmetry = Symmetry.orthorhombic
    lattice_material = Lattice.orthorhombic(a, b, c)
elif symmetry_name == "tetragonal":
    crystal = SGLattice(int(SG),a, c)
    symmetry = Symmetry.tetragonal
    lattice_material = Lattice.tetragonal(a, c)
elif symmetry_name == "trigonal":
    crystal = SGLattice(int(SG),a, alpha)
    symmetry = Symmetry.trigonal
    lattice_material = Lattice.rhombohedral(a, alpha)
elif symmetry_name == "triclinic":
    crystal = SGLattice(int(SG),a, b, c, alpha, beta, gamma)
    symmetry = Symmetry.triclinic
    lattice_material = Lattice.triclinic(a, b, c, alpha, beta, gamma)

symmetry1 = None
lattice_material1 = None
crystal1 = None

lattice_, lattice1_ = lattice_material, lattice_material1

## load model related files and generate the model
json_file = open(model_direc+"//model_Al2TiO5.json", 'r')

classhkl = np.load(model_direc+"//MOD_grain_classhkl_angbin.npz")["arr_0"]
angbins = np.load(model_direc+"//MOD_grain_classhkl_angbin.npz")["arr_1"]

ind_mat = None
ind_mat1 = None  

load_weights = model_direc + "//model_Al2TiO5.h5"
wb = read_hdf5(load_weights)
temp_key = list(wb.keys())

# # load json and create model
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
print("Constructing model")
model.load_weights(load_weights)
print("Uploading weights to model")
print("All model files found and loaded")

ct = time.time()
now = datetime.datetime.fromtimestamp(ct)
c_time = now.strftime("%Y-%m-%d_%H-%M-%S")


hkl_all_class1 = None
with open(model_direc+"//classhkl_data_nonpickled_"+material_+".pickle", "rb") as input_file:
    hkl_all_class0 = cPickle.load(input_file)[0]


_file = open(detectorfile, "r")
text = _file.readlines()
_file.close()
# first line contains parameters
parameters = [float(elem) for elem in str(text[0]).split(",")]
detectorparameters = parameters[:5]
pixelsize = parameters[5]
dim1 = parameters[6]
dim2 = parameters[7]
# others are comments
comments = text[1:]
ccd_label_global = ""
for line in comments:
    if line.startswith("# CCDLabel"):
        ccd_label_global = line.split(":")[1].strip()
if ccd_label_global == "":
    print("CCD label cannot be read from the calibration file, setting it to latest detector sCMOS")
    ccd_label_global = "sCMOS"


col = [[] for i in range(int(ubmat))]
colx = [[] for i in range(int(ubmat))]
coly = [[] for i in range(int(ubmat))]
rotation_matrix = [[] for i in range(int(ubmat))]
strain_matrix = [[] for i in range(int(ubmat))]
strain_matrixs = [[] for i in range(int(ubmat))]
match_rate = [[] for i in range(int(ubmat))]
spots_len = [[] for i in range(int(ubmat))]
iR_pix = [[] for i in range(int(ubmat))]
fR_pix = [[] for i in range(int(ubmat))]
mat_global = [[] for i in range(int(ubmat))]
best_match = [[] for i in range(int(ubmat))]
spots1_global = [[] for i in range(int(ubmat))]
for i in range(int(ubmat)):
    col[i].append(np.zeros((lim_x*lim_y,3)))
    colx[i].append(np.zeros((lim_x*lim_y,3)))
    coly[i].append(np.zeros((lim_x*lim_y,3)))
    rotation_matrix[i].append(np.zeros((lim_x*lim_y,3,3)))
    strain_matrix[i].append(np.zeros((lim_x*lim_y,3,3)))
    strain_matrixs[i].append(np.zeros((lim_x*lim_y,3,3)))
    match_rate[i].append(np.zeros((lim_x*lim_y,1)))
    spots_len[i].append(np.zeros((lim_x*lim_y,1)))
    iR_pix[i].append(np.zeros((lim_x*lim_y,1)))
    fR_pix[i].append(np.zeros((lim_x*lim_y,1)))
    mat_global[i].append(np.zeros((lim_x*lim_y,1)))
    best_match[i].append([[] for jk in range(lim_x*lim_y)])
    spots1_global[i].append([[] for jk in range(lim_x*lim_y)])


if use_previous_UBmatrix_name:
    np.savez_compressed(model_direc+'//rotation_matrix_indexed_1.npz', rotation_matrix, mat_global, match_rate, 0.0)

# =============================================================================
#         ## Multi-processing routine
# =============================================================================        
## Number of files to generate
grid_files = np.zeros((lim_x,lim_y))
filenm = np.chararray((lim_x,lim_y), itemsize=1000)
grid_files = grid_files.ravel()
filenm = filenm.ravel()
count_global = lim_x * lim_y

format_file = dictLT.dict_CCD[ccd_label_global][7]

list_of_files = glob.glob(filenameDirec+'//'+experimental_prefix+'*.'+format_file)
## sort files
## TypeError: '<' not supported between instances of 'str' and 'int'
list_of_files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

if len(list_of_files) == count_global:
    for ii in range(len(list_of_files)):
        grid_files[ii] = ii
        filenm[ii] = list_of_files[ii]               
else:
    print("expected "+str(count_global)+" files based on the XY grid ("+str(lim_x)+","+str(lim_y)+") defined by user")
    print("But found "+str(len(list_of_files))+" files (either all data is not written yet or maybe XY grid definition is not proper)")
    digits = len(str(count_global))
    digits = max(digits,4)
    # Temp fix
    for ii in range(count_global):
        text = str(ii)
        if ii < 10000:
            string = text.zfill(4)
        else:
            string = text.zfill(5)
        file_name_temp = filenameDirec+'//'+experimental_prefix + string+'.'+format_file
        ## store it in a grid 
        filenm[ii] = file_name_temp

check = np.zeros((count_global,int(ubmat)))
# =============================================================================
blacklist = None

### Create a COR directory to be loaded in LaueTools
cor_file_directory = filenameDirec + "//" + experimental_prefix+"CORfiles"
if not os.path.exists(cor_file_directory):
    os.makedirs(cor_file_directory)

try_prevs = False
files_treated = []
mode_spotCycle = "graphmode_allHKL"

valu12 = [[filenm[ii].decode(), ii,
           rotation_matrix,
            strain_matrix,
            strain_matrixs,
            col,
            colx,
            coly,
            match_rate,
            spots_len, 
            iR_pix, 
            fR_pix,
            best_match,
            mat_global,
            check,
            detectorparameters,
            pixelsize,
            angbins,
            classhkl,
            hkl_all_class0,
            hkl_all_class1,
            emin,
            emax,
            material_,
            material1_,
            symmetry,
            symmetry1,   
            lim_x,
            lim_y,
            strain_calculation, 
            ind_mat, ind_mat1,
            model_direc, float(tolerance),
            float(tolerance1),
            int(ubmat), ccd_label_global, 
            None,
            float(intensity_threshold),
            int(boxsize),bkg_treatment,
            filenameDirec, 
            experimental_prefix,
            blacklist,
            None,
            files_treated,
            try_prevs, ## try previous is kept true, incase if its stuck in loop
            wb,
            temp_key,
            cor_file_directory,
            mode_spotCycle,
            softmax_threshold_global,
            mr_threshold_global,
            cap_matchrate,
            tolerance_strain,
            tolerance_strain1,
            NumberMaxofFits,
            fit_peaks_gaussian,
            FitPixelDev,
            coeff,
            coeff_overlap,
            material0_limit,
            material1_limit,
            use_previous_UBmatrix_name,
            material_phase_always_present,
            crystal,
            crystal1] for ii in range(count_global)]

#import threading
#timermp1212 = threading.Timer(1, update_data_mp)
#timermp1212.start()


#chunks = chunker_list(valu12, ncpu)
#chunks_mp = list(chunks)
#_inputs_queue = Queue()
#_outputs_queue = Queue()
#run_flag = multip.Value('I', True)
#_worker_processes = {}
#for i in range(ncpu):
#    _worker_processes[i]= Process(target=worker, args=(_inputs_queue, _outputs_queue, i+1, run_flag))
#for i in range(ncpu):
#    _worker_processes[i].start()   
#meta = {'t1':time.time()}
#for ijk in range(int(ncpu)):
#    _inputs_queue.put((chunks_mp[ijk], ncpu, meta))
    

Constructing model
Uploading weights to model
All model files found and loaded


In [ ]:
def new_MP_function(files, cnt, rotation_matrix, strain_matrix, strain_matrixs,\
                col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,mat_global,\
                check,detectorparameters,pixelsize,angbins,\
                classhkl, hkl_all_class0, hkl_all_class1, emin, emax,\
                material_, material1_, symmetry, symmetry1,lim_x,lim_y,\
                strain_calculation, ind_mat, ind_mat1,\
                model_direc, tolerance , tolerance1,\
                matricies, ccd_label,\
                filename_bkg,intensity_threshold,\
                boxsize,bkg_treatment,\
                filenameDirec, experimental_prefix,\
                blacklist_file, text_file, \
                files_treated,try_previous1,\
                wb, temp_key, cor_file_directory, mode_spotCycle1,\
                softmax_threshold_global123,mr_threshold_global123,\
                cap_matchrate123, tolerance_strain123, tolerance_strain1231,\
                NumberMaxofFits123,fit_peaks_gaussian_global123,\
                FitPixelDev_global123,coeff123,coeff_overlap,\
                material0_limit, material1_limit, use_previous_UBmatrix_name1,\
                    material_phase_always_present1, crystal, crystal1):
    print("working")            
    global total_progress
    total_progress += 1
    progress.value = total_progress/max_progress     

    try:
        strain_matrix12, strain_matrixs12, \
            rotation_matrix12, col12, \
                colx12, coly12,\
        match_rate12, mat_global12, cnt12,\
            files_treated12, spots_len12, \
                iR_pix12, fR_pix12, check12, best_match12 = predict_preprocessMultiProcess(files, cnt, 
                                                   rotation_matrix,strain_matrix,strain_matrixs,
                                                   col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,
                                                   mat_global,
                                                   check,detectorparameters,pixelsize,angbins,
                                                   classhkl, hkl_all_class0, hkl_all_class1, emin, emax,
                                                   material_, material1_, symmetry, symmetry1,lim_x,lim_y,
                                                   strain_calculation, ind_mat, ind_mat1,
                                                   model_direc, tolerance, tolerance1,
                                                   matricies, ccd_label,
                                                   filename_bkg,intensity_threshold,
                                                   boxsize,bkg_treatment,
                                                   filenameDirec, experimental_prefix,
                                                   blacklist_file, text_file, 
                                                   files_treated,try_previous1,
                                                   wb, temp_key, cor_file_directory, mode_spotCycle1,
                                                   softmax_threshold_global123,mr_threshold_global123,
                                                   cap_matchrate123, tolerance_strain123,
                                                   tolerance_strain1231,NumberMaxofFits123,
                                                   fit_peaks_gaussian_global123,
                                                   FitPixelDev_global123, coeff123,coeff_overlap,
                                                   material0_limit,material1_limit,
                                                   use_previous_UBmatrix_name1,
                                                   material_phase_always_present1,
                                                   crystal, crystal1)
        return True, strain_matrix12, strain_matrixs12, rotation_matrix12, col12, \
                     colx12, coly12, match_rate12, mat_global12, cnt12, meta, \
                     files_treated12, spots_len12, iR_pix12, fR_pix12, best_match12, check12
    except Exception as e:
        return False, np.zeros((3,3))


if __name__ == "__main__":

    display(progress)
    
    import multiprocessing 
    
    with multiprocessing.Pool(ncpu) as pool:
        results = [pool.apply_async(new_MP_function, p) for p in valu12]
        for r in results:
            r_message_mpdata = r.get()
            if r_message_mpdata[0] == False:
                continue

            strain_matrix_mpdata, strain_matrixs_mpdata, rotation_matrix_mpdata, col_mpdata, \
                             colx_mpdata, coly_mpdata, match_rate_mpdata, mat_global_mpdata, \
                                 cnt_mpdata, meta_mpdata, files_treated_mpdata, spots_len_mpdata, \
                                     iR_pixel_mpdata, fR_pixel_mpdata, best_match_mpdata, check_mpdata = r_message_mpdata

            for i_mpdata in files_treated_mpdata:
                files_treated.append(i_mpdata)

            for intmat_mpdata in range(int(ubmat)):
                check[cnt_mpdata,intmat_mpdata] = check_mpdata[cnt_mpdata,intmat_mpdata]
                mat_global[intmat_mpdata][0][cnt_mpdata] = mat_global_mpdata[intmat_mpdata][0][cnt_mpdata]
                strain_matrix[intmat_mpdata][0][cnt_mpdata,:,:] = strain_matrix_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                strain_matrixs[intmat_mpdata][0][cnt_mpdata,:,:] = strain_matrixs_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                rotation_matrix[intmat_mpdata][0][cnt_mpdata,:,:] = rotation_matrix_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                col[intmat_mpdata][0][cnt_mpdata,:] = col_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                colx[intmat_mpdata][0][cnt_mpdata,:] = colx_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                coly[intmat_mpdata][0][cnt_mpdata,:] = coly_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                match_rate[intmat_mpdata][0][cnt_mpdata] = match_rate_mpdata[intmat_mpdata][0][cnt_mpdata]
                spots_len[intmat_mpdata][0][cnt_mpdata] = spots_len_mpdata[intmat_mpdata][0][cnt_mpdata]
                iR_pix[intmat_mpdata][0][cnt_mpdata] = iR_pixel_mpdata[intmat_mpdata][0][cnt_mpdata]
                fR_pix[intmat_mpdata][0][cnt_mpdata] = fR_pixel_mpdata[intmat_mpdata][0][cnt_mpdata]
                best_match[intmat_mpdata][0][cnt_mpdata] = best_match_mpdata[intmat_mpdata][0][cnt_mpdata]

    print("Launched all processes")  

FloatProgress(value=0.0, max=1.0)